In [ ]:
# rodar o melhor modelo do tempo para predição de SST e firmeza e montar tabela

In [2]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

%load_ext autoreload

sys.path.append(os.getcwd()+'\\..\\scripts\\')
path = os.getcwd() + '\\..\\'

from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGFeaturesSize import MNGFeaturesSize
from MNGModel import MNGModel

In [54]:
data = pd.read_csv(path+'features\\means_size_all_half.csv', sep=';', index_col=0)
palmer_sst = pd.read_csv(path+'palmer_sst.csv', sep=';')
palmer_firm = pd.read_csv(path+'palmer_firmeza.csv', sep=';')

n_trees = 200
kf = KFold(n_splits=5, shuffle=True)
data['sst'] = palmer_sst.values
data['firmeza'] = palmer_firm.values

X = data.drop(columns=['time', 'sst', 'firmeza'])
Y_sst = data['sst']
Y_firm = data['firmeza']
Y_time = data['time']

predictions = pd.DataFrame(index=data.index.values, columns=['sst_predict', 'firmeza_predict', 'time_predict'])
real = pd.DataFrame(index=data.index.values, columns=['sst_real', 'firmeza_real', 'time_real'])

In [59]:
for train_i, test_i in kf.split(X):
    X_train, X_test = X.iloc[train_i], X.iloc[test_i]
    Y_sst_train, __ = Y_sst.iloc[train_i], Y_sst.iloc[test_i]
    Y_firm_train, __ = Y_firm.iloc[train_i], Y_firm.iloc[test_i]
    Y_time_train, __ = Y_time.iloc[train_i], Y_time.iloc[test_i]
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_sst_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['sst_predict'].iloc[test_i] = Y_predicted
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_firm_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['firmeza_predict'].iloc[test_i] = Y_predicted
    
    rf_model = RandomForestRegressor(n_estimators=n_trees).fit(X_train, Y_time_train)
    Y_predicted = rf_model.predict(X_test)
    predictions['time_predict'].iloc[test_i] = Y_predicted

In [55]:
real['sst_real'] = palmer_sst.values
real['firmeza_real'] = palmer_firm.values
real['time_real'] = data['time']

In [99]:
predictions.to_csv('dados_predict.csv', sep=';')
predictions['sst_real'] = real['sst_real']
predictions['firmeza_real'] = real['firmeza_real']
predictions['time_real'] = data['time']

predictions.to_csv('dados_predict_reais.csv', sep=';')

In [95]:
def calc_tabela(data, att, range_max, range_min, intervals, consider_end):
    indexes = []
    bins = math.floor((range_max-range_min)/intervals)
    tabela = pd.DataFrame(data=0,index=np.arange(bins),columns=data.columns)
    
    for bin_num in range(bins):
        min_v = range_min if bin_num == 0 else range_min + intervals*bin_num + 1
        if bin_num!=(bins-1):
            max_v = min_v + intervals if bin_num == 0 else min_v + intervals - 1
        else:
            max_v = range_max
        indexes.append(str(min_v)+'-'+str(max_v))
        tabela.iloc[bin_num] = data[(min_v<=data[att]) & (data[att]<=max_v)].mean()
    
    tabela.index = indexes
    return tabela

In [85]:
range_max = 170
range_min = 35
intervals = 15

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, False)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, False)

In [96]:
range_max = 190
range_min = 171
intervals = 9

tabela_real = calc_tabela(real, 'time_real', range_max, range_min, intervals, True)
tabela_pred = calc_tabela(predictions, 'time_predict', range_max, range_min, intervals, True)